# Suite2p Activity Trace and Behavioral Data Converter


## What does this script do:

Section 1) Converts Suite2p extracted ROI traces and neuropil data output into a numpy or csv file ready for downstream event-related analysis

Section 2) Create a pickle file containing a dictionary where each key represents a condition/event-type in the session and the corresponding values are lists containing time stamps of each event in samples

How to run this code
------------------------------------

In this jupyter notebook, First find the code block with the comment header called USER-DEFINED VARIABLES. Edit the variables according to your data and output preferences. Then just run all cells in order (shift + enter; or in the dropdown menu: Kernel->Resart & Run All).

User-Defined Parameters 
----------

fdir : string 

    Root file directory containing the data files. IMPORTANT Note: leave off the last backslash, and include the letter r in front of string (treats the contents as a raw string). For example: r'C:\Users\my_user\analyze_sessions'

fname : string

    Session name; by default this is the name of the parent folder that the data resides in, but can be changed by user to be any string. This fname variable is mainly used to name the saved output files.

In [ ]:
import numpy as np
import pickle as pkl
import os
import pandas as pd
import s2p_con_func

In [ ]:
"""
USER DEFINE VARIABLES

fdir: string
    Directory where the root raw data resides in (not the suite2p output folder)
    for your own data, please refer to guidance on formatting in above markdown documentation
    replace the contents inside the apostrophes with the path to your data; make sure the r comes before the apostrophe

fname: string
    Root name of the raw data (without file extensions). Will ultimately dictate the base name of the saved files
    If left alone, it will use the parent folder name
"""

fdir = os.path.abspath('./sample_data/VJ_OFCVTA_7_260_D6_snippit')
fname = os.path.split(fdir)[-1]



In [ ]:
if __name__=="__main__":
    threshold_scaling_values = 0
    
    s2p_con_func.csv_npy_save(fdir, fname)